In [1]:
import cv2
from ultralytics import YOLO
import time
import sp_modules as sp
import numpy as np

# Load the YOLOv8 model
model = YOLO('./weights/20240106_best_weight_yolov8_640.pt')

# Open the video file
# file_path = 'E:/Raw data/datasets/AVATAR/'
# file_name = 'Hab_01_1200f'
file_path = 'D:/spkim/data/20241114_40Hz_test/20241114_40Hz_test/'
file_name = 'C_20Hz_Bandi_20fps'
input_video_name = file_path + file_name + '.mp4'
output_video_name = file_path + file_name + '_output.mp4'

cap = cv2.VideoCapture(input_video_name)

# Define the codec and create VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
fps = cap.get(cv2.CAP_PROP_FPS)  # Use the same FPS as the source video
frame_total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Use the total frame to estimate the time
frame_width = int(cap.get(3))  # Width of the frames in the video
frame_height = int(cap.get(4))  # Height of the frames in the video
frame_width_resize = 800
frame_height_resize = 600

out = cv2.VideoWriter(output_video_name, fourcc, fps, (frame_width, frame_height))
# out = cv2.VideoWriter(output_video_name, fourcc, fps, (frame_width_resize, frame_height_resize))
current_frame=0

# Color code setting for center-point
color_code = {
    "red": (0, 0, 255),
    "orange": (0, 165, 255),
    "yellow": (0, 255, 255),
    "green": (0, 128, 0),
    "blue": (255, 0, 0),
    "skyblue": (235, 206, 135),
    "purple": (128, 0, 128),
    "black": (0, 0, 0),
    "pink": (255, 192, 203)
}
color_order = ["red", "orange", "yellow", "green", "blue", "skyblue", "purple", "black", "pink"]
output = []

#To match darknet (YOLOv4), #4, 5: forelimb / #6, 7: hindlimb
rearrange = [4, 6, 0, 1, 2, 8, 3]
# Five areas from captured frame (x1, y1, x2, y2)
area = [
    [ 1, 1, 1200, 1000],
    [ 1201, 1, 2400, 1000],
    [ 2401, 1, 3600, 1200],
    [ 1, 1001, 1200, 2000],
    [ 1201, 1001, 2400, 2000]
]
raw_coordinates = [0] * 135
while cap.isOpened():
    start_time = time.time() # Time measurement
    success, frame = cap.read()
    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame, verbose=False)
                
        # Visualize the results on the frame
        # annotated_frame = results[0].plot(line_width = 1, labels = False)
        annotated_frame = frame
        
        # Draw bounding box and center
        check_cls = [[0 for _ in range(10)] for _ in range(10)]
        raw_coordinates = [0] * 135
        
        for det in results[0].boxes:
            # det: [x1, y1, x2, y2, conf, cls]
            xy = det.xyxy.tolist()[0]
            conf, cls = det.conf.item(), int(det.cls.item())
            # cls definition
            # 0: fore / 1: hind / 2: nose / 3: head / 4: ass / 5: tail / 6: torso

            # Calculate the center of the bounding box
            center_x, center_y = (int(xy[0]) + int(xy[2])) // 2, (int(xy[1]) + int(xy[3])) // 2
            
            # Get only one xy pair (highest conf value) in each camera view
            for i in range(5):
                # Find the ROI (camera 0 to 4)
                if(center_x > area[i][0] and center_x < area[i][2] and center_y > area[i][1] and center_y < area[i][3]):
                    # forehand
                    if(cls == 0 or cls == 1): # 0: fore / 1: hind
                        if(check_cls[i][rearrange[cls]] == 0):
                            raw_coordinates[i*27 + rearrange[cls]*3] = center_x
                            raw_coordinates[i*27 + 1 + rearrange[cls]*3] = center_y
                            raw_coordinates[i*27 + 2 + rearrange[cls]*3] = conf
                            check_cls[i][rearrange[cls]] += 1
                        elif(check_cls[i][rearrange[cls]] == 1):
                            raw_coordinates[i*27 + rearrange[cls]*3 + 3] = center_x
                            raw_coordinates[i*27 + 1 + rearrange[cls]*3 + 3] = center_y
                            raw_coordinates[i*27 + 2 + rearrange[cls]*3 + 3] = conf
                            check_cls[i][rearrange[cls]] += 1
                    elif(check_cls[i][rearrange[cls]] == 0): # Rest of body points
                        raw_coordinates[i*27 + rearrange[cls]*3] = center_x
                        raw_coordinates[i*27 + 1 + rearrange[cls]*3] = center_y
                        raw_coordinates[i*27 + 2 + rearrange[cls]*3] = conf
                        check_cls[i][rearrange[cls]] += 1

            # Optionally draw the bounding box
            cv2.rectangle(annotated_frame, (int(xy[0]), int(xy[1])), (int(xy[2]), int(xy[3])), color=(255, 0, 0), thickness=2)
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break
    
    # Measure an angle of each limb from body-ass line (4, 5: forelimb / 6, 7: hindlimb / 2: body / 3: ass)

    # forelimb (54 is for bottom camera)
    f1_angle = sp.get_AB_AC_angle(raw_coordinates[54 + 3*3], raw_coordinates[54 + 3*3 +1],
                               raw_coordinates[54 + 2*3], raw_coordinates[54 + 2*3 +1],
                               raw_coordinates[54 + 4*3], raw_coordinates[54 + 4*3 +1])

    f2_angle = sp.get_AB_AC_angle(raw_coordinates[54 + 3*3], raw_coordinates[54 + 3*3 +1],
                               raw_coordinates[54 + 2*3], raw_coordinates[54 + 2*3 +1],
                               raw_coordinates[54 + 5*3], raw_coordinates[54 + 5*3 +1])
    
    if(f1_angle < 180 and f2_angle > 180):        
        temp = raw_coordinates[54 + 4*3 : 54 + 4*3 +2]
        raw_coordinates[54 + 4*3: 54 + 4*3 +2] = raw_coordinates[54 +5*3 : 54 +5*3 +2]
        raw_coordinates[54 + 5*3 : 54+ 5*3 +2] = temp
    
    # hindlimb
    h1_angle = sp.get_AB_AC_angle(raw_coordinates[54 + 3*3], raw_coordinates[54 + 3*3+1],
                               raw_coordinates[54 + 2*3], raw_coordinates[54 + 2*3+1],
                               raw_coordinates[54 + 6*3], raw_coordinates[54 + 6*3+1])

    h2_angle = sp.get_AB_AC_angle(raw_coordinates[54 + 3*3], raw_coordinates[54 + 3*3+1],
                                   raw_coordinates[54 + 2*3], raw_coordinates[54 + 2*3+1],
                                   raw_coordinates[54 + 7*3], raw_coordinates[54 + 7*3+1])
    
    if(h1_angle < 180 and h2_angle > 180):
        temp = raw_coordinates[54 + 6*3 : 54 + 6*3+2]
        raw_coordinates[54 + 6*3 : 54 + 6*3+2] = raw_coordinates[54 +7*3 : 54 + 7*3+2]
        raw_coordinates[54 + 7*3 : 54 + 7*3+2] = temp

    # Draw a center point of each feature
    for i in range(5):
        for j in range(9):
            if(raw_coordinates[i*27+j*3] != 0):
                cv2.circle(annotated_frame, (raw_coordinates[i*27+j*3], raw_coordinates[i*27+j*3+1]),
                        radius=10, color=color_code[color_order[j]], thickness=-1)

    # Display the frame with bounding box and center
    # resized_frame = cv2.resize(annotated_frame, (800, 600))
    out.write(annotated_frame)
    # cv2.imshow("YOLOv8 Inference", resized_frame)

    # Write the annotated frame to the output video file
    out.write(annotated_frame)
    
    # Result accumulation
    output.append(raw_coordinates)
    
    # Print total estimated time
    sp.print_remaining_time(start_time, time.time()-start_time, current_frame, frame_total)
    
    current_frame += 1

# Print job completion
if(current_frame == frame_total):
    print('\r', flush=True) # Make sure the blank
    print("Pose estimation complete!", end='\r', flush=True)

# Save to txt file
with open(file_path+file_name+'.txt', 'w') as file:
    for row in output:
        file.write('\t'.join(map(str, row)) + '\n')

cap.release()
out.release()
cv2.destroyAllWindows()

208 of 209 frames processed: 0.00 mins left
Pose estimation complete!